# DBS Readout Model
## Discrete Network

A lot of recent effort in DBS research has focused on identifying neural signals that track with disease state.
These *disease readouts* can objectively inform whether DBS is working and whether therapy changes are appropriate.
Not a lot of effort has focused on developing a systematic framework to finding these readouts, however.

In this notebook we'll develop a theoretical approach to designing and assessing neural readouts for diseases.

In [1]:
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as sig
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import scipy.stats as stats
import sklearn.linear_model as linear_model

import seaborn as sns
sns.set_style("white")
matplotlib.rcParams['figure.figsize'] = [15, 10]

### Our model
We start with a simple Graph $\mathcal{G}$.
A disease mapping $\Xi \in \mathbb{R}^D$ transforms from the neural statespace to the behavioral statespace.
A measurement mapping $h: X \rightarrow Y$ yields us our electrophysiologic measurements $\vec{y}(t)$.
The goal of a readout is to accurately predict the behavioral state $\beta$ from just the measurements $\vec{y}(t)$.

### Signal Model
Our output state $z$ is a convex combination of $x$ and $y$.
$\alpha$ is the free parameter that determines the ratio of $x$ to $y$ in the output signal $z$.

noise_z is the noise level of the measurement $\hat{z} =  z + \mathcal{N}$.
We then calculate the correlation between $z$ and our known salient input $x$.

\begin{equation}
    z(t) = \alpha \cdot x(t) + (1-\alpha) \cdot y(t) \\
    \hat{z}(t) = z + \mathcal{N}
\end{equation}


In [2]:
bi_net = nx.Graph()
bi_net.add_nodes_from([1,2,3])
bi_net.add_edge(1,3,weight=10)
bi_net.add_edge(2,3,weight=1)

In [3]:
def gen_sig(fs=100,noise_x=1.0,noise_y=5.0,alpha=1.,fc_x=0.5,fc_y=0.5,noise_z=1):
    t = np.linspace(0,100,100*fs)
    input_x = 1
    input_y = 2
    
    phase_x = np.random.uniform(-np.pi,np.pi,100*fs)
    phase_y = np.random.uniform(-np.pi,np.pi,100*fs)
    
    x = np.random.normal(np.sin(2 * np.pi * (fc_x+ input_x) * t + phase_x),noise_x)
    y = np.random.normal(np.sin(2 * np.pi * (fc_y+input_y) * t + phase_y),noise_y)
    z = alpha*x + (1-alpha)*y + np.random.normal(0,noise_z,size=x.shape)
    
    fig = plt.figure(figsize=(14,12))
    plt.subplot(1,2,1)
    nx.draw(bi_net)
    
    plt.subplot(1,2,2)
    
    plt.plot(t,z,color='green',alpha=0.8)
    plt.plot(t,y,alpha=1-alpha)
    plt.plot(t,x,alpha=alpha)
    
    
    plt.ylim((-10,10))
    
    pears = stats.pearsonr(z,x)
    spears = stats.spearmanr(z,x)
    plt.title('Correlation $\hat{z}$ w/ x: ' + str(pears[0]))
    
#interactive(gen_sig,fs=(1,100,1),noise_x=(0.0,1.,0.1),noise_y=(0.0,1.,0.1),alpha=(0.0,1,0.01),samps=(1,1000,10),noise_z=(0.0,5.,0.1),fc_x=(0.01,5.,0.05),fc_y=(0.01,5.,0.05))

In [4]:
interactive(gen_sig,fs=fixed(100),noise_x=fixed(0.5),noise_y=fixed(0.5),alpha=(0.0,1,0.01),noise_z=(0.0,5.,0.1),fc_x=fixed(4.5),fc_y=fixed(4.7))

interactive(children=(FloatSlider(value=1.0, description='alpha', max=1.0, step=0.01), FloatSlider(value=1.0, …

Next, we'll explore the role that noise in our bullshit node ($Y$) interferes with our ability to predict $X$.

In [5]:
interactive(gen_sig,fs=fixed(100),noise_x=fixed(0.5),noise_y=(0.0,10,0.1),alpha=(0.0,1,0.01),noise_z=(0.0,5.,0.1),fc_x=fixed(4.5),fc_y=fixed(4.7))

interactive(children=(FloatSlider(value=5.0, description='noise_y', max=10.0), FloatSlider(value=1.0, descript…

## Effect of blending on correlation coefficient
In this example we'll explore the role of 'blending' between two neural signals into a measurement.

We start with three brain nodes: $x$, $w$, and $z$.
Our *measurement* $y = \alpha x + (1-\alpha) w$.
Our *behavior* $\beta = \gamma x + (1-\gamma) z$.

### How good are our measurements
In our measurement, we've got some activity from a node that is involved in behavior $x$ but also some activity from a node that is not related to behavior $z$.
We then have a *signal-to-noise* ratio in our recording $y$ that we can calculate.

\begin{align}
\text{SNR} = \frac{E[\alpha x^2]}{E[(1-alpha) w^2]} = \frac{\alpha}{(1-\alpha)} \cdot \frac{\sigma_x}{\sigma_w}
\end{align}

In [6]:
t = np.linspace(0,10,100)

x = np.sin(t) + np.random.normal(0,2,size=t.shape)
w = np.random.normal(0,2,size=t.shape)


def snr(alpha,w_var=2):
    y = alpha * x + (1-alpha) * w
    plt.figure()
    plt.subplot(211)
    plt.plot(x,alpha=alpha+0.1)
    plt.plot(w,alpha=1-alpha+0.1)
    plt.plot(y)
    plt.subplot(212)
    plt.scatter(x,y)
    plt.xlim((-5,5))
    plt.ylim((-5,5))
    
    corr_val = stats.pearsonr(y,x)
    spear_val = stats.spearmanr(y,x)
    slope = linear_model.LinearRegression(fit_intercept=True).fit(y.reshape(-1,1),x.reshape(-1,1)).coef_[0]
    plt.title(str(corr_val[0]) + ' ' + str(spear_val[0]) + ' Slope:' + str(slope))

interact(snr,alpha=(0.0,1.0,0.01),w_var=fixed(2))

interactive(children=(FloatSlider(value=0.5, description='alpha', max=1.0, step=0.01), Output()), _dom_classes…

<function __main__.snr(alpha, w_var=2)>

## Perfect Model
In this example we'll work with a 'perfect' model of our system.

We start with a set of brain regions $\{x_i\}_i$

We'll have our disease output $c = \beta^i \cdot x_i$

And we'll have our measurement $y = m^i \cdot x_i + \epsilon$

In [7]:
def overlap(b7=0.15,b8=0.15,m1=1, m2=1, m3=1,h_noise=1,disease_mask=np.array([1,1,1,1,1,1,1,1,1,1])):
    G = nx.Graph()
    G.add_nodes_from(np.arange(10))
    
    
    # Set up our disease map
    beta = np.multiply(np.array([0.1,0.1,0.1,0.1,0.1,0.1,b7,b8,0.0,0.0]),disease_mask)
    beta = beta/np.linalg.norm(beta)

    # Set up our measurement map
    h = np.array([0,0,0,0,0,0,m1,m2,m3,1])
    h = h/np.linalg.norm(h,ord=1)

    # How many observations do we have
    trials = 10000

    # What's the noise in each brain region
    x_sigma = 1.0
    x = np.random.normal(0,x_sigma,size=(10,trials))

    # Now calculate our disease measure and recordings
    c = np.dot(beta,x).reshape(-1,1)
    y = np.dot(h,x).reshape(-1,1)

    # Add in noise to our recording if we want to
    y += np.random.normal(0,h_noise,size=y.shape)

    max_ip = np.dot(c.T,y)
    max_corr = stats.pearsonr(c.squeeze(),y.squeeze())
    
    regr = linear_model.LinearRegression().fit(c,y)
    slope = regr.coef_[0]
    
    plt.figure()
    plt.plot([-1,1],[-1,1])
    plt.scatter(c,y)
    plt.xlim((-5,5))
    plt.ylim((-5,5))
    plt.title('Pearson is: ' + str(np.round(max_corr[0],4)) + ' Slope: ' + str(slope))
    

In [8]:
overlapper = interactive(overlap,m1=(0,1.0,0.05),m2=(0,1.0,0.05),m3=(0,1.0,0.05),b7=(0.0,10.0,0.05),b8=(0.0,10.0,0.05),h_noise=(0.0,10.0,0.5),disease_mask=fixed(np.array([1,1,1,1,1,1,1,1,1,1])))
display(overlapper)

#print(h)

#print(1/trials * max_ip/(np.std(c)*np.std(y)))

interactive(children=(FloatSlider(value=0.15, description='b7', max=10.0, step=0.05), FloatSlider(value=0.15, …

Let's take a look at our model so far

![](Assets/discrete_model_labels.png)

## How good is good enough?

What we see in the above example is that even if Node 7 + Node 8 contribute to $\frac{2.0}{2.6} \approx 77\%$.

We've put a limit on how good a readout can be compared to perfect.
Here we'll see how we can go about figuring out if a readout is *good enough*

## Predicted vs Actual
We'll play around with our Predicted vs Actual plots now


In [9]:
disease_mask = np.array([0,0,0,0,0,0,1.0,1.0,0.0,0.0])

In [10]:
def simpl_model(noise_level=0,hidden_region=0.0,extra_brain=0.0):
    x = np.random.uniform(-1,1,(100,6))
    b = np.dot(x,np.array([1.0,-1.0,hidden_region,0.0,0.0,0.0]))
    y = np.dot(x,np.array([[1.0,0.0,extra_brain,0.0,0.0,0.0],[0.0,1.0,0.0,0.0,0.0,0]]).T)
    y += np.random.normal(0,noise_level,size=y.shape)
    #regress y to b
    reg = linear_model.LinearRegression().fit(y,b)
    print(reg.coef_)

    x_test = np.random.uniform(-1,1,(50,6))
    b_test = np.dot(x_test,np.array([1.0,-1.0,0.0,0.0,0.0,0.0]))
    y_test = np.dot(x_test,np.array([[1.0,0.0,0.0,0.0,0.0,0.0],[0.0,1.0,0.0,0.0,0.0,0]]).T)
    y_test += np.random.normal(0,noise_level,size=y_test.shape)
    pred_b = reg.predict(y_test)
    score = reg.score(y_test,b_test)

    pear = stats.pearsonr(b_test,pred_b)

    pva = linear_model.LinearRegression().fit(b_test.reshape(-1,1),pred_b.reshape(-1,1))

    plt.figure()
    plt.scatter(b_test,pred_b)
    plt.title('Pear:' + str(pear) + ' Slope:' + str(pva.coef_[0]) + ' Score:' + str(score))

sys = interactive(simpl_model,noise_level=(0.0,5.0,0.1),hidden_region=(0.0,2.0,0.1),extra_brain=(0.0,1.0,0.1))
display(sys)


interactive(children=(FloatSlider(value=0.0, description='noise_level', max=5.0), FloatSlider(value=0.0, descr…